In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import betabinom


def compute_beta(alpha, n, mean):
    return (1-mean/n) / (mean/n) * alpha


def compute_alpha(n, mean, variance):
    ratio = (1-mean/n) / (mean/n)
    alpha = ((1+ratio)**2 * variance - n**2 * ratio) / (n*ratio*(1+ratio) - variance* (1 + ratio)**3)
    return alpha

In [ ]:
if __name__ == "__main__":
    np.random.seed(19680801)

    # example data
    n = 30  # Size of the support (i.e. numbers between 0 and n)
    mu = 10  # mean of distribution
    sigma = 4  # standard deviation of distribution
    
    # Compute the alpha and beta for the given mu and sigma
    alpha = compute_alpha(n , mu, sigma**2)
    beta = compute_beta(alpha, n, mu)
    
    x = betabinom.rvs(n, alpha, beta, size=100000)

    num_bins = n+1

    fig, ax = plt.subplots()

    # the histogram of the data
    n, bins, patches = ax.hist(x, num_bins, density=True)

    ax.set_xlabel('Number of outliers')
    ax.set_ylabel('Probability density')
    ax.set_title(r'Histogram of Beta-binominal dist: $\mu=8$, $\sigma=4$')

    # Tweak spacing to prevent clipping of ylabel
    fig.tight_layout()
    plt.show()

In [1]:
import os
import sys
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/")
import argparse
import numpy as np
import networkx as nx
import slam.plot as splt
import slam.topology as stop
import slam.generate_parametric_surfaces as sgps
import trimesh
import os
import tools.graph_processing as gp
from sphere import *
from tqdm.auto import tqdm,trange
from scipy.stats import betabinom
import random
import pickle 

In [2]:
def generate_sphere_random_sampling(vertex_number=100, radius=1.0):
    """
	generate a sphere with random sampling
	:param vertex_number: number of vertices in the output spherical mesh
	:param radius: radius of the output sphere
	:return:
	"""
    coords = np.zeros((vertex_number, 3))
    for i in range(vertex_number):
        M = np.random.normal(size=(3, 3))
        Q, R = np.linalg.qr(M)
        coords[i, :] = Q[:, 0].transpose() * np.sign(R[0, 0])
    if radius != 1:
        coords = radius * coords
    return coords

def tri_from_hull(vertices):
    """
	compute faces from vertices using trimesh convex hull
	:param vertices: (n, 3) float
	:return:
	"""
    mesh = trimesh.Trimesh(vertices=vertices, process=False)
    return mesh.convex_hull


def edge_len_threshold(graph,thr): # Adds a percentage of edges 
    
    edge_to_add = random.sample(list(graph.edges),round(len(graph.edges)*thr))

    return edge_to_add

In [3]:
def compute_beta(alpha, n, mean):
    return (1-mean/n) / (mean/n) * alpha


def compute_alpha(n, mean, variance):
    ratio = (1-mean/n) / (mean/n)
    alpha = ((1+ratio)**2 * variance - n**2 * ratio) / (n*ratio*(1+ratio) - variance* (1 + ratio)**3)
    return alpha


def generate_nb_outliers_and_nb_supress(nb_vertices):

    """

    Sample nb_outliers and nb_supress from a Normal dist
    following the std of real data

    """

    mean_real_data = 10         # mean real data
    std_real_data = 4           # std real data


    mu = 10 # mu_A = mu_B = mu
    sigma = std_real_data
    n = 20

    alpha = compute_alpha(n , mu, sigma**2)  # corresponding alpha with respect to given mu and sigma
    beta = compute_beta(alpha, n, mu)       # corresponding beta

    nb_supress = betabinom.rvs(n, alpha, beta, size=1)[0]
    nb_outliers = betabinom.rvs(n, alpha, beta, size=1)[0]                 # Sample nb_outliers


    return int(nb_outliers),int(nb_supress)

In [280]:
def generate_noisy_graph(original_graph, nb_vertices, sigma_noise_nodes=1400, sigma_noise_edges=1,
                         radius=100):
    # Perturbate the coordinates

    noisy_coord = []
    key = []
    value = []
    nb_outliers = 0
    nb_supress = 0

    for index in range(nb_vertices):
        # Sampling from Von Mises - Fisher distribution
        original_coord = original_graph.nodes[index]["coord"]
        mean_original = original_coord / np.linalg.norm(original_coord)  # convert to mean unit vector.
        noisy_coordinate = Sphere().sample(1, distribution='vMF', mu=mean_original,
                                           kappa=sigma_noise_nodes).sample[0]

        noisy_coordinate = noisy_coordinate * np.linalg.norm(original_coord) # rescale to original size.
        # print(noisy_coordinate)
        noisy_coord.append(noisy_coordinate)

        print(original_coord," ",noisy_coordinate)


    nb_outliers, nb_supress = generate_nb_outliers_and_nb_supress(nb_vertices)  # Sample nb_outliers and nb_supress
    #nb_outliers = 0
    
    noisy_coord_all = noisy_coord
    

    #Supress Non-Outlier nodes
    if nb_supress > 0:
        
        supress_list = random.sample(range(len(noisy_coord)), nb_supress) # Indexes to remove 
        
        removed_coords = [noisy_coord[i] for i in range(len(noisy_coord)) if i in supress_list]
        
        #noisy_coord = [dummy_coords if i in supress_list else noisy_coord[i] for i in range(len(noisy_coord))]
        
        noisy_coord = [noisy_coord[i] for i in range(len(noisy_coord)) if i not in supress_list]
        
        print("nb_supress : ",nb_supress)
        
    print("nb_outliers : ",nb_outliers)
        
            
    #print('Noisy coord len after supression',len(noisy_coord_all))
    #print('removed coord len after supression',len(removed_coords))


    # Add Outliers
    sphere_random_sampling = []
    if nb_outliers > 0:
        #print("nb_outliers: ", nb_outliers)

        sphere_random_sampling = generate_sphere_random_sampling(vertex_number=nb_outliers, radius=radius)
        # merge pertubated and outlier coordinates to add edges 
        all_coord = noisy_coord + list(sphere_random_sampling)
    else:
        all_coord = noisy_coord


    noisy_graph = nx.Graph()

    compute_noisy_edges = tri_from_hull(all_coord)  # take all peturbated coord and comp conv hull.
    adja = stop.adjacency_matrix(compute_noisy_edges)  # compute the new adjacency mat.

    noisy_graph = nx.from_numpy_matrix(adja.todense())
    

    node_attribute_dict = {}
    for node in noisy_graph.nodes():
        node_attribute_dict[node] = {"coord": np.array(compute_noisy_edges.vertices[node]),'is_dummy':False,'is_outlier':False}
        
#     dummy_nodes = [j for j in range(len(noisy_graph.nodes), len(noisy_graph.nodes) + len(supress_list))] 
#     noisy_graph.add_nodes_from(dummy_nodes,is_dummy=True) # add dummy nodes
    
        
    nx.set_node_attributes(noisy_graph, node_attribute_dict)
    nx.set_edge_attributes(noisy_graph, 1.0, name="weight")

    
    edge_attribute_dict = {}
    id_counter = 0  # useful for affinity matrix caculation
    for edge in noisy_graph.edges:
        # We calculate the geodesic distance
        end_a = noisy_graph.nodes()[edge[0]]["coord"]
        end_b = noisy_graph.nodes()[edge[1]]["coord"]
        geodesic_dist = gp.compute_geodesic_distance_sphere(end_a, end_b, radius)

        # add the information in the dictionnary
        edge_attribute_dict[edge] = {"geodesic_distance": geodesic_dist, "id": id_counter}
        id_counter += 1

    # add the edge attributes to the graph
    nx.set_edge_attributes(noisy_graph, edge_attribute_dict)

    # Extracting the ground-truth correspondence

    
    ground_truth_permutation = []
    counter = 0
    check = False
#     print(dummy_nodes)
    
    for i in range(len(noisy_graph.nodes)): 
        for j in range(len(noisy_coord_all)):  # upto the indexes of outliers
            if np.linalg.norm(noisy_coord_all[j] - noisy_graph.nodes[i]['coord']) == 0.:
                ground_truth_permutation.append(j)
                continue
                
            elif j == len(noisy_coord_all) - 1.:
                 for outlier in sphere_random_sampling:
                        if np.linalg.norm(outlier - noisy_graph.nodes[i]['coord']) == 0.:
                            noisy_graph.nodes[i]['is_outlier'] = True
                            ground_truth_permutation.append('O')
          
        
#         if check == False:
#             print(counter)
            
#             if dummy_nodes[counter] >= len(noisy_coord_all):
#                 key.append(i)
#             ground_truth_permutation.append(dummy_nodes[counter])#add idx of dummy node as correspondence
#             counter += 1
#         check = False
                
            

#     for outlier in sphere_random_sampling:
#         for i in range(len(all_coord)):
#             if np.mean(noisy_graph.nodes[i]['coord']) == np.mean(outlier):
#                 if i<nb_vertices:
#                     value.append(i)


#     if nb_outliers > 0 and len(key)!=0:
#         index = 0
#         for j in range(len(ground_truth_permutation)):
#             if ground_truth_permutation[j] == key[index]:
#                 ground_truth_permutation[j] = value[index]
#                 index+=1
#                 if index == len(key):
#                     break

#         key = key + value
#         value = value + key

#         mapping = dict(zip(key,value))
#         #print("mapping :",mapping)
#         #print("number of nodes in graphs: ", len(noisy_graph.nodes))
#         noisy_graph = nx.relabel_nodes(noisy_graph, mapping)


    # Remove 10% of random edges
#     edge_to_remove = edge_len_threshold(noisy_graph, 0.10)
#     noisy_graph.remove_edges_from(edge_to_remove)

#     noisy_graph.remove_edges_from(nx.selfloop_edges(noisy_graph))


    return ground_truth_permutation, noisy_graph,noisy_coord_all

In [281]:
list_graphs= []

In [282]:
ref_graph = nx.read_gpickle('reference_1.gpickle')

In [283]:
ground_truth, noisy_graph, noisy_coord_all = generate_noisy_graph(ref_graph,20)

[ 20.96346116 -76.88340545 -60.41088695]   [ 18.85659395 -77.90242492 -59.79666426]
[-40.38208717  36.61107826  83.83863062]   [-39.98447318  40.06027737  82.44037895]
[-53.14616772  26.3147491  -80.51719591]   [-55.73982188  27.1302843  -78.46668039]
[87.30901573 46.98534633 13.01971589]   [86.75154354 48.69062733 10.16821047]
[-50.93066397  85.66396199  -8.23122617]   [-47.0820403   87.62970463 -10.21353749]
[-85.9712732   50.983334    -3.10480883]   [-84.92910496  52.51129854  -5.44156738]
[-83.8724879  -23.10325171 -49.31171803]   [-86.58194857 -21.6051973  -45.13071716]
[ -8.14699669  97.86962485 -18.84576812]   [ -6.3257007   98.02440791 -18.74035657]
[-90.38310799  25.9637659   34.01141941]   [-89.62284852  28.40550632  34.07157517]
[ 2.31108465 83.28236347 55.30557678]   [ 0.40383906 81.48609734 57.96423771]
[84.6487079  14.32457521 51.27770271]   [84.38524166 11.4527751  52.42103521]
[ 81.88240267 -57.22654565   4.51603881]   [ 82.64875213 -56.23394244   2.63201231]
[ 20.71161

In [284]:
ground_truth

[18, 'O', 10, 15, 2, 'O', 'O', 1, 'O', 'O', 14, 5, 16, 'O', 'O', 13, 17, 19]

In [152]:
ref_graph.nodes[0]

{'coord': array([ 20.96346116, -76.88340545, -60.41088695])}

In [191]:
def get_in_between_perm_matrix(perm_mat_1, perm_mat_2):
    """
	Given two permutation from noisy graphs to a reference graph,
	Return the permutation matrix to go from one graph to the other
	"""
    result_perm = {}
    for i in range(len(perm_mat_1)):
        if perm_mat_1[i] == 'O':
                continue
        for j in range(len(perm_mat_2)):
            if perm_mat_2[j] == 'O':
                continue
            
            if perm_mat_1[i] == perm_mat_2[j]:
                result_perm[i] = j

    return result_perm

In [192]:
gt_0 = ground_truth_0
gt_1 = ground_truth_1
# gt_2 = ground_truth_2
# gt_3 = ground_truth_3
gt_all = [gt_0,gt_1]

In [193]:
new_groundtruth = []

In [194]:
result_perm = get_in_between_perm_matrix(gt_0,gt_1)

In [195]:
result_perm

{2: 7, 5: 6, 7: 17, 10: 22, 12: 14, 17: 26, 18: 19, 21: 16}

In [196]:
for i,j in result_perm.items():
    print(noisy_graph_0.nodes[i]['coord'],'----',noisy_graph_1.nodes[j]['coord'])

[87.96029962 43.95165181 18.19994488] ---- [87.90662198 46.27881099 11.43229917]
[ 82.9968391  -55.77855793   0.52647356] ---- [ 81.17070132 -57.9376393    7.38560761]
[82.64595281 17.32215314 53.56854949] ---- [84.93741475 14.36852202 50.78563921]
[ 21.41644424 -77.66644408 -59.23900218] ---- [ 21.91153331 -78.10018138 -58.4828725 ]
[ 47.52387721 -69.9761136   53.33689737] ---- [ 49.82963337 -69.76524611  51.47638364]
[ 69.85966807 -62.38129156 -35.04570217] ---- [ 70.35443955 -64.42147507 -30.00210637]
[ 69.83559898  66.13268484 -27.37621433] ---- [ 72.3575313   65.1184683  -22.89045107]
[ 24.54262499  34.17415789 -90.71817068] ---- [ 20.4184669   31.78021106 -92.59106001]


In [ ]:
gt_03 = {}

for i in range(len(gt_0)):
    for j in range(len(gt_3)):
        if gt_0[i] == gt_3[j]:
            gt_03[i] = j

In [ ]:
new_groundtruth = []
new_groundtruth = [gt_00,gt_01,gt_02,gt_03]

In [ ]:
ground_dict = {}

for i in range(1):
    for j in range(4):
        ground_dict[str(i)+str(j)] =globals()['gt_'+str(i)+str(j)] 

In [ ]:
ground_dict

In [ ]:
for key,value in gt_02.items():
    print(noisy_graph.nodes[key]['coord'],noisy_graph_2.nodes[value]['coord'])

In [ ]:
# for key,value in g3_dict.items():
#     if value == -1:
#         continue
#     else:
#         print(noisy_graph_3.nodes[key]['coord'],'-----',value,'----',noisy_graph.nodes[value]['coord'])

In [ ]:
for i in range(len(ref_graph.nodes)):
    print(ref_graph.nodes[i]['coord'],noisy_coord_all[i])

In [ ]:
for i in range(len(ref_graph.nodes)):
    print(ref_graph.nodes[i]['coord'],noisy_coord_all_3[i])

In [ ]:
# nx.read_gpickle('test_graphs.gpickle')

In [ ]:
# np.load('new_groundtruth.npy',allow_pickle=True)

In [ ]:
test_graphs = [noisy_graph,noisy_graph_1,noisy_graph_2,noisy_graph_3]
np.save('new_groundtruth.npy',new_groundtruth)
nx.write_gpickle(test_graphs,'test_graphs.gpickle')

In [ ]:
noisy_graph_2.nodes

In [ ]:
print('g3 node attr len: ',nx.get_node_attributes(g1,'label_gt'))

In [ ]:
for i,j in enumerate(ground_truth_permutation):
    print(noisy_graph.nodes[i]['coord'],noisy_coord_all[j])

In [ ]:
ground_truth_permutation

In [ ]:
shuffled_coords

In [ ]:
ground_truth

In [ ]:
noisy_coord_all[3]

In [ ]:
noisy_graph.nodes[12]

In [ ]:
for i,j in enumerate(ground_truth):
    
    if noisy_graph.nodes[j]['is_dummy'] == False:
        
        print(noisy_graph.nodes[j]['coord'],ref_graph.nodes[i]['coord'])

In [ ]:
rm = random.sample(range(10), 5)
rm

In [ ]:
ground_truth

In [ ]:
np.sort(ground_truth)

In [ ]:
# [ground_truth[i] for i in range(len(ground_truth)) if i not in rm]

In [ ]:
[ref_graph.nodes[i]['coord'] for i in range(len(ref_graph.nodes)) if i in rm]

In [ ]:
noisy_coords = list(nx.get_node_attributes(ref_graph,'coord').values())

dummy_coords = np.array([0.0,0.0,0.0])

noisy_coords_2 = [dummy_coords if i in rm else noisy_coords[i] for i in range(len(noisy_coords))]

In [ ]:
# nx.get_node_attributes(noisy_graph,'coord')

In [ ]:
# noisy_graph.add_nodes_from([(13, {'is_dummy': True}),(14, {'is_dummy': True}),(15, {'is_dummy': True})])

In [ ]:
# noisy_graph.add_node(16,is_dummy=True)

In [ ]:
ground_truth

In [ ]:
# for i in ground_truth:
#     print(noisy_graph.nodes[i])

In [ ]:
gt = np.load('ground_truth.npy')

In [ ]:
# path_graph = "./noise_1000,outliers_20/graphs/"

In [ ]:
# g0 = nx.read_gpickle(path_graph+'graph_12.gpickle')
# g1 = nx.read_gpickle(path_graph+'graph_1.gpickle')
# g8 = nx.read_gpickle(path_graph+'graph_8.gpickle')

In [ ]:
# gt_12 = {}

# for i in range(len(gt_1)):
#     for j in range(len(gt_2)):
#         if gt_1[i] == gt_2[j]:
#             gt_12[i] = j
            

In [ ]:
# for i,j in gt_12.items():
#     print(noisy_graph.nodes[i]['coord'], noisy_graph_1.nodes[j]['coord'])

In [ ]:
# for i,j in enumerate(ground_truth_1):
#     print(noisy_graph_1.nodes[i]['coord'],noisy_coord_all[j])

In [ ]:
g_updated = nx.read_gpickle('g_updated.gpickle')
g1_updated = nx.read_gpickle('g1_updated.gpickle')
g2_updated = nx.read_gpickle('g2_updated.gpickle')
g3_updated = nx.read_gpickle('g3_updated.gpickle')



g_values = nx.get_node_attributes(g1_updated,'label_gt').values()

for i,j in enumerate(g_values):
    
    if(j == -1):
        continue
        
    else:
        print(g_updated.nodes[j]['coord'],g1_updated.nodes[i]['coord'])
        print(g_updated.nodes[j]['label_gt'],'....',g1_updated.nodes[i]['label_gt'])

In [ ]:
list_graphs = [len(g_updated.nodes),len(g1_updated.nodes),len(g2_updated.nodes),len(g3_updated.nodes)]

In [ ]:
list_graphs.index(min(list_graphs))

In [ ]:
list_graphs

In [ ]:
g1_updated.nodes[4]['label_gt']

In [ ]:
g1_updated.nodes.data()

In [ ]:
#check if there is -1 in attribute

#number of nodes to add in the graph

In [ ]:
gt_label = list(nx.get_node_attributes(g1_updated,'label_gt').values())
count_val = gt_label.count(-1)
num_nodes_to_add = len(g_updated.nodes())-(len(g1_updated.nodes)-count_val)

In [ ]:
num_nodes_to_add

In [ ]:
np.sort(gt_label)

In [ ]:
list(g_updated.nodes)

In [ ]:
len_g1 = len(g1_updated.nodes)

labels = list(set(list(g_updated.nodes)).symmetric_difference(set(gt_label)))

if -1 in labels:
    labels.remove(-1)

In [ ]:
labels

In [ ]:
for i,j in enumerate(labels):
    
    g1_updated.add_node(len_g1+i,label_gt=j,coord=[-0.0,-0.0,-0.0])

In [ ]:
np.sort(list(nx.get_node_attributes(g1_updated,'label_gt').values()))

In [ ]:
# def dummy_for_visu(g,g1):
    
#     gt_label = list(nx.get_node_attributes(g1,'label_gt').values())
    
#     len_g1 = len(g1.nodes)

    
#     labels = list(set(list(g.nodes)).symmetric_difference(set(gt_label)))

#     if -1 in labels:
#         labels.remove(-1)
        
#     for i,j in enumerate(labels):
        
#         g1.add_node(len_g1+i,label_gt=j,coord=[-0.0,-0.0,-0.0])

In [ ]:
dummy_for_visu(g_updated,g1_updated)

In [ ]:
list(nx.get_node_attributes(g1_updated,'label_gt').values())

In [387]:
graph_0 = nx.read_gpickle('./noise_800,outliers_20/graphs/graph_00000.gpickle')
graph_1 = nx.read_gpickle('./noise_800,outliers_20/graphs/graph_00001.gpickle')
graph_2 = nx.read_gpickle('./noise_800,outliers_20/graphs/graph_00002.gpickle')
graph_3 = nx.read_gpickle('./noise_800,outliers_20/graphs/graph_00003.gpickle')
# graph_5 = nx.read_gpickle('./noise_1400,outliers_20/graphs/graph_5.gpickle')

grt_01 = np.load('./noise_800,outliers_20/ground_truth.gpickle',allow_pickle=True)

In [388]:
nx.number_of_nodes(graph_2)

17

In [389]:
new_groundtruth = [grt_01['0,0'], grt_01['0,1'], grt_01['0,2'], grt_01['0,3']]

In [390]:
test_graphs = [graph_0,graph_1,graph_2,graph_3]
np.save('new_groundtruth.npy',new_groundtruth)
nx.write_gpickle(test_graphs,'test_graphs.gpickle')

In [391]:
new_groundtruth

[{2: 2, 5: 5, 10: 10, 11: 11, 12: 12, 14: 14},
 {2: 16, 5: 15, 10: 3, 11: 5, 14: 14},
 {5: 12, 12: 10},
 {5: 18, 10: 10, 11: 13, 14: 16}]

In [394]:
for i,j in new_groundtruth[1].items():
    print(graph_0.nodes[i]['coord'],'...',graph_1.nodes[j]['coord'])
    print(graph_0.nodes[i]['is_outlier'],'...',graph_1.nodes[j]['is_outlier'])

[-94.56324843  31.94520655   6.10703079] ... [-94.85703263  31.63370855  -1.20492514]
False ... False
[-48.60981682 -52.42534082  69.91901993] ... [-56.25603427 -51.17239957  64.93569227]
False ... False
[ 93.82162054  11.55761375 -32.61786448] ... [ 93.77491742  11.834926   -32.65270878]
False ... False
[-60.32108386   7.78459544 -79.3773703 ] ... [-61.88796636  14.42646136 -77.21241372]
False ... False
[-15.59387718 -39.16153913 -90.6818882 ] ... [ -9.41205102 -41.07835039 -90.68617549]
False ... False


In [ ]:
new_groundtruth[1]

In [ ]:
for key,value in new_groundtruth[1].items():
    
    print(graph_0.nodes[key]['coord'], graph_1.nodes[value]['coord'])

In [ ]:
graph_1.nodes

In [ ]:
new_groundtruth[2].items()

In [ ]:
gt_02 = grt_01['02']
g0 = graph_0.copy()
g2 = graph_2.copy()

outliers_label = -1

dict_lab = {}
for n, node in enumerate(g2.nodes):
    if node not in list(gt_02.values()):
        dict_lab[node] = {'label_gt':outliers_label}
        
    else:
        
        dict_lab[node] = {'label_gt':list(gt_02.keys())[list(gt_02.values()).index(node)]}

In [210]:
gt = './noise_1500,outliers_20/ground_truth.gpickle'
gt = nois

In [206]:
np.sort(os.listdir('./noise_1500,outliers_20/graphs/'))

array(['graph_00000.gpickle', 'graph_00001.gpickle',
       'graph_00002.gpickle', 'graph_00003.gpickle',
       'graph_00004.gpickle', 'graph_00005.gpickle',
       'graph_00006.gpickle', 'graph_00007.gpickle',
       'graph_00008.gpickle', 'graph_00009.gpickle',
       'graph_00010.gpickle', 'graph_00011.gpickle',
       'graph_00012.gpickle', 'graph_00013.gpickle',
       'graph_00014.gpickle', 'graph_00015.gpickle',
       'graph_00016.gpickle', 'graph_00017.gpickle',
       'graph_00018.gpickle', 'graph_00019.gpickle'], dtype='<U19')

In [ ]:
def get_permutation_matrix_from_dictionary(matching, g_sizes):
    """
    Create the full permutation matrix from the matching result
    :param matching: the matching result for each graph (nodes number, assignment)
    :param g_sizes: the list of the size of the different graph
    :return: the full permutation matrix
    """
    f_size = int(np.sum(g_sizes))
    res = np.zeros((f_size, f_size))

    idx1 = 0
    for i_g1 in range(len(sizes)):
        idx2 = 0
        for i_g2 in range(len(sizes)):
            match = matching["{},{}".format(i_g1, i_g2)]
            for k in match:
                res[idx1 + int(k), idx2 + match[k]] = 1
            idx2 += sizes[i_g2]
        idx1 += sizes[i_g1]

    return res

In [345]:
path_to_graphs = './noise_1500,outliers_20/graphs/'
path_to_groundtruth_ref = './noise_1500,outliers_20/permutation_to_ref_graph.gpickle'
reference_graph = nx.read_gpickle('reference_0.gpickle')

In [308]:
list_graphs = load_graphs_in_order(path_to_graphs)
ground_truth = nx.read_gpickle(path_to_groundtruth)

In [263]:
sizes = [nx.number_of_nodes(g) for g in list_graphs]

In [333]:
nodes = [list(graph.nodes()) for graph in list_graphs]

In [330]:
for graph,perm in zip(list_graphs,ground_truth):
    
    for i,j in enumerate(perm):
        
        if j == 'O':
            continue
            
        print(graph.nodes[i]['coord'],reference_graph.nodes[j]['coord'])


[-97.80548343  -8.96316602  18.80821806] [-98.0701938  -11.20343727  16.02248674]
[-43.60572913   3.98506594 -89.90361303] [-43.05288179   2.70645786 -90.21709625]
[ 68.44258321  71.68983814 -13.2732781 ] [ 66.5622926   73.40626273 -13.45294748]
[-47.6285131  -42.24344192  77.11690058] [-46.71281266 -38.95646934  79.37447089]
[34.39975188 70.04587704 62.53184932] [34.85079114 71.50510136 60.60068346]
[-83.40981704 -54.79472549  -6.35141564] [-82.56146591 -55.96585618  -7.17128217]
[-14.73654717 -46.24582237 -87.43087607] [-14.52337439 -44.90820666 -88.16078817]
[-29.12658875 -69.72282329  65.50091404] [-27.99677138 -69.06989269  66.67481321]
[99.80204108 -5.15639509  3.60058126] [99.80901978 -4.71223371  3.99429903]
[-59.97308298 -24.3009728  -76.24101284] [-61.20794245 -21.99989099 -75.9578342 ]
[ 72.16803969 -56.56706062  39.8991441 ] [ 70.55795154 -57.47179196  41.45562211]
[ -9.10369379 -85.83046767 -50.50003544] [ -5.23776152 -82.58417386 -56.14641647]
[ 20.70765473 -53.35939405  

In [338]:
ground_truth[9]

[22, 'O', 6, 0, 11, 20, 26, 'O', 1, 16, 7, 10]

In [339]:
from graph_generation.load_graphs_and_create_metadata import dataset_metadata

In [385]:
graph_meta = dataset_metadata(path_to_graphs,path_to_groundtruth_ref)
graph_meta.labels
graph_meta.sizes

[22,
 19,
 18,
 20,
 10,
 27,
 20,
 22,
 17,
 25,
 26,
 19,
 24,
 25,
 27,
 24,
 24,
 24,
 19,
 31]

In [ ]:
graph_meta.list_graphs